In [2]:
from src.argilla.server.daos.backend.generic_elastic import GenericElasticEngineBackend

In [3]:
elastic_backend = GenericElasticEngineBackend.get_instance()

In [4]:
elastic_backend.client

ElasticsearchClient(vector_search_supported=True, index_shards=1, config_backend={'hosts': 'http://localhost:9200', 'verify_certs': True, 'ca_certs': None, 'retry_on_timeout': True, 'max_retries': 5})

In [ ]:
elastic_backend.create_datasets_index()